In [1]:
import torch
import numpy as np
import torchvision

## Design a Distribution of Prediction with Softmax
In the output layer, we need a softmax layer, the sum of each category is 1:
1. $P(y = i) \ge 0$
2. $\sum P(y = i) = 1$

Suppose $Z^l \in R^k$ is the output of the last linear layer, the softmax function:
$$P(y = i) = \frac{e^{z_i}}{\sum_{j=0}^{K-1}e^{z_j}}, i \in \{0, \dots, K-1\}$$

### The loss function - Cross Entropy
$$Loss(\hat{Y}, Y) = -Y\log(\hat{Y})$$

In [2]:
## Implenmentation
import numpy as np
y = np.array([1, 0, 0])
z = np.array([0.2, 0.1, -0.1])
y_pred = np.exp(z)/np.exp(z).sum()
loss = (-y * np.log(y_pred)).sum()
print(loss)

0.9729189131256584


In [3]:
import torch
y = torch.LongTensor([0])
z = torch.Tensor([[0.2, 0.1, -0.1]])
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(z, y)
print(loss)

tensor(0.9729)


In [5]:
import torch
criterion = torch.nn.CrossEntropyLoss()
Y = torch.LongTensor([2, 0, 1]) # prediction of 3 batches
Y_pred1 = torch.Tensor([[0.1, 0.2, 0.9], # 2
                       [1.1, 0.1, 0.2], # 0 
                       [0.2, 2.1, 0.1]]) # 1
Y_pred2 = torch.Tensor([[0.8, 0.2, 0.3], # 2 
                       [0.2, 0.3, 0.5], # 0
                       [0.2, 0.2, 0.5]]) # 1
l1 = criterion(Y_pred1, Y)
l2 = criterion(Y_pred2, Y)
print("Batch Loss1 = ",l1.data, "\nBatch Loss2= ", l2.data)

Batch Loss1 =  tensor(0.4966) 
Batch Loss2=  tensor(1.2389)


In [6]:
# Prepare Dataset
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim


In [7]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081, ))
])

train_dataset = datasets.MNIST(root='../dataset/mnist/',
                               train=True,
                               download=True,
                               transform=transform)
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/',
                              train = False,
                              download=True,
                              transform=transform)
test_loader = DataLoader(test_dataset,
                         shuffle=False,
                         batch_size=batch_size)

100%|██████████| 9912422/9912422 [00:02<00:00, 4483433.34it/s]


Extracting ../dataset/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 7716140.76it/s]

Extracting ../dataset/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3028725.34it/s]


Extracting ../dataset/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17461529.58it/s]

Extracting ../dataset/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw



In [ ]:
## Design Model
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

In [ ]:
model = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.5)

In [ ]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader):
        inputs, target = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss/300))
            running_loss = 0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_dataset:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %d %%' % (100 * correct / total))

In [ ]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()